In [1]:
# | default_exp wb_clim

In [2]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [31]:
# | export
import collections
import numpy as np
from typing import Dict
from pandera.typing import DataFrame
from sureau_ecos_py.climate_utils import compute_vpd_from_t_rh, day_length

from sureau_ecos_py.create_simulation_parameters import (
    create_simulation_parameters,
)

from sureau_ecos_py.create_climate_data import create_climate_data
from sureau_ecos_py.create_modeling_options import create_modeling_options

In [4]:
# | export


def new_wb_clim(
    climate_data: DataFrame,  # Dataframe created using the `create_climate_data` function
    year: int,  # Year, __No definition found__
    day_of_year: int,  # Day of the year
) -> Dict:  # Dictionary containing parameters to run the model
    "Create a list with all necessary daily climate values to run SureauR from climate_data"

    # Assert parameters ---------------------------------------------------------

    # Year
    assert (
        isinstance(year, int) and 3000 >= year > 0
    ), "year must be a integer value between 0-3000"

    # Day of year
    assert (
        isinstance(day_of_year, int) and 366 >= day_of_year >= 1
    ), "day_of_year must be a integer value between 1-366"

    # Create wb_clim dictionary -------------------------------------------------

    # Check that year and day_of_year are present inside the dataframe
    if (
        year in climate_data["year"].values
        and day_of_year in climate_data["day_of_year"].values
    ):
        # Get row index in climate frame based on year and doy
        row_index = climate_data[
            (climate_data["year"] == year)
            & (climate_data["day_of_year"] == day_of_year)
        ].index[0]

        # Transfrom row to a dictionary with params
        wb_clim_dict = collections.defaultdict(
            list, dict(climate_data.loc[row_index])
        )

    else:
        raise ValueError(
            f"year:{year} and/or day_of_year:{day_of_year} not found in climate dataframe"
        )

    # Add parameters to dictionary ----------------------------------------------
    wb_clim_dict["net_radiation"] = float("NAN")
    wb_clim_dict["etp"] = float("NAN")
    wb_clim_dict["vpd"] = compute_vpd_from_t_rh(
        relative_humidity=wb_clim_dict["RHair_mean"],
        temperature=wb_clim_dict["Tair_mean"],
    )

    # Add Temperature from previous and next days

    # cas normal

    # if the row_index is not the first nor the last
    if row_index != 0 and row_index != climate_data.shape[0]:
        wb_clim_dict["Tair_min_prev"] = climate_data.loc[row_index - 1][
            "Tair_min"
        ]
        wb_clim_dict["Tair_min_next"] = climate_data.loc[row_index + 1][
            "Tair_min"
        ]
        wb_clim_dict["Tair_max_prev"] = climate_data.loc[row_index - 1][
            "Tair_max"
        ]

        return wb_clim_dict

    # si premier jour de le la simu

    # if the row_index is the first
    elif row_index == 0:
        print(
            "Firts day of the simulation, previous Tair is the same as the current"
        )

        wb_clim_dict["Tair_min_prev"] = climate_data.loc[row_index]["Tair_min"]
        wb_clim_dict["Tair_min_next"] = climate_data.loc[row_index + 1][
            "Tair_min"
        ]
        wb_clim_dict["Tair_max_prev"] = climate_data.loc[row_index]["Tair_max"]

        return wb_clim_dict

    elif row_index == climate_data.shape[0]:
        print(
            "Last day of the simulation, next Tair_min_next is the same as the Tair_min"
        )

        wb_clim_dict["Tair_min_prev"] = climate_data.loc[row_index - 1][
            "Tair_min"
        ]
        wb_clim_dict["Tair_min_next"] = climate_data.loc[row_index]["Tair_min"]
        wb_clim_dict["Tair_max_prev"] = climate_data.loc[row_index - 1][
            "Tair_max"
        ]

        return wb_clim_dict

    else:
        raise ValueError(
            "Error setting previous and following temperature conditions"
        )

#### __Example: Create wb_clim dictionary__

In [5]:
simulation_parameters_dict = create_simulation_parameters(
    main_dir="./sample_data",
    start_year_simulation=1991,
    end_year_simulation=1991,
    output_type=None,
    output_path="./sample_data",
    overwrite=True,
    resolution_output="subdaily",
)

In [6]:
modeling_options_dict = create_modeling_options(
    time_step_for_evapo=2,
    reset_swc=True,
    avoid_water_soil_transfer=True,
    constant_climate=False,
    defoliation=True,
    soil_evapo=True,
    threshold_mortality=51,
    etp_formulation="pt",
    rn_formulation="linear",
    comp_options_for_evapo="custom",
    stomatal_reg_formulation="turgor",
    transpiration_model="jarvis",
    numerical_scheme="implicit",
    pedo_transfer_formulation="vg",
)

In [7]:
climate_dataframe = create_climate_data(
    modeling_options=modeling_options_dict,
    simulation_parameters=simulation_parameters_dict,
    file_path="./sample_data/climat_example.csv",
    sep=";",
)

365 days were selected in the input climate file, covering the period: 1991 - 1991


In [8]:
new_wb_clim(climate_data=climate_dataframe, year=1991, day_of_year=365)

Last day of the simulation, next Tair_min_next is the same as the Tair_min


defaultdict(list,
            {'DATE': Timestamp('1991-12-31 00:00:00'),
             'Tair_min': -0.2,
             'Tair_max': 2.7,
             'Tair_mean': 1.25,
             'RG_sum': 0.71,
             'RHair_min': 96.9151449,
             'RHair_max': 96.9151449,
             'RHair_mean': 96.9151449,
             'WS_mean': 1.7,
             'PPT_sum': 0.0,
             'day_of_year': 365,
             'day_of_month': 31,
             'month': 12,
             'year': 1991,
             'net_radiation': nan,
             'etp': nan,
             'vpd': 0.020627738121622427,
             'Tair_min_prev': 2.4,
             'Tair_min_next': -0.2,
             'Tair_max_prev': 4.4})

In [34]:
# | export


def new_wb_clim_hour(
    wb_clim: Dict,  # Dictionary created using the `new_wb_clim` function
    wb_veg: Dict,  # __No definition found__
    latitude: float,  # Value indicating the latitude of the stand
    longitude: float,  # Value indicating the longitude of the stand
    modeling_options: Dict,  # Dictionary created using the `create_modeling_options` function
    pt_coeff: float,  # An empirical constant accounting for the vapor pressure deficit and resistance values Typically, α is 1.26 for open bodies of water, but has a wide range of values from less than 1 (humid conditions) to almost 2 (arid conditions).
) -> Dict:  # Dictionary containing parameters to run the model
    "Create a list with interpolated climate data at the required time step"

    # Assert parameters ---------------------------------------------------------

    # wb_clim
    assert isinstance(
        wb_clim, Dict
    ), f"wb_clim must be a Dictionary not a {type(wb_clim)}"

    # wb_veg
    assert isinstance(
        wb_veg, Dict
    ), f"wb_veg must be a Dictionary not a {type(wb_veg)}"

    # modeling_options
    assert isinstance(
        modeling_options, Dict
    ), f"modeling_options must be a Dictionary not a {type(modeling_options)}"

    # Latitude and longitude
    assert (
        isinstance(latitude, float) and isinstance(longitude, float)
    ), "Missing latitude and/or longitude. Provide latitude and/or longitude as Coordinates points i.e. latitude = 41.40338, longitude = 2.17403"

    # pt_coeff
    assert isinstance(
        pt_coeff, float
    ), f"pt_coeff must be a float i.e. 2.0001 not a {type(pt_coeff)}"

    # Create wb_clim_hour dictionary --------------------------------------------
    wb_clim_hour = collections.defaultdict(list)

    if modeling_options["constant_climate"] is False:
        # calculate sunrise, sunset and daylength (in seconds from midgnight)
        # depends of DAY, latt and lon
        # sunrise_sunset_daylen <- as.numeric(daylength(lat = lat, long = lon, jd = WBclim$DOY, 0)) * 3600 #

        # Calculate day_length
        sunrise_sunset_daylength_hours = day_length(latitude = latitude,
                                                     day_of_year = wb_clim['day_of_year'])

        # Transform day_length to seconds
        sunrise_sunset_daylength_seconds = collections.defaultdict(list)

        for each_key, each_array in sunrise_sunset_daylength_hours.items():
            sunrise_sunset_daylength_seconds[each_key].append(each_array*3600)


#### __Example:__

defaultdict(<class 'list'>, {'sunrise': array([99., 99., 99.]), 'sunset': array([99., 99., 99.]), 'day_length': array([24., 24., 24.])})


/workspaces/sureau_ecos_py/sureau_ecos_py/climate_utils.py:298: UserWarning: Double check if this function works for Australia
  warnings.warn("Double check if this function works for Australia")


defaultdict(list,
            {'sunrise': [array([356400., 356400., 356400.])],
             'sunset': [array([356400., 356400., 356400.])],
             'day_length': [array([86400., 86400., 86400.])]})